# LecoSpec Data Munging

In [5]:
# notebooks use their location as their working directory, so
# if we are in a subfolder, move to the main folder.  
# This however can safely be run multiple times
#setwd(M:/lecospec/lecospec)
if(!dir.exists("Functions/")){
    setwd("../")
    if(!dir.exists("Functions")){
        setwd("M:/lecospec/lecospec/")
    }
}
source("Functions/lecospectR.R", echo = FALSE)

## Notation

Throughout the notebook, variables starting with `img_` are UAV image-based information (data, filepaths, etc).  Similarly, variables beginning with `grd_` related to data collected on the ground.  

Also, some other naming conventions for variables with data transformations:
* `robust` in a variable name refers to data treated by center according to the median and scaling by teh inter-quartile range (a la sklearns RobustScaler)
* `minmax` (and its ilk) are min-max scaled data, i.e. scaled to the interval [0,1] by subtracting the minimum and dividing by the range.
* `standard(ized)` refers to data treated with with the z-score transform by centring using the mean and scaling y the standard deviation (like sklearns StandardScaler)
* `corrected` means that a linear transformation has been applied to account for differences in sensor calibration.
* `raw` refers to having no transformations applied
* `clipped` means that outliers have been clipped to the upper and lower fence values based on the Inter-Quartile Range method. 
* `imputed` means that outliers have been removed and imputed
* `dropped` means that dataframe rows containing outliers have been removed

Example: `img_robust_indices` refers to vegetation indices from the UAV images treated with the robust scaler. 

## Define data locations


In [2]:
# spectral library
grd_base_path <- "./Output/C_001_SC3_Cleaned_SpectralLib.csv"
grd_speclib <- read.csv(grd_base_path, header = TRUE)
#grd_index_path <- ./Data/D_002_SpecLib_Derivs.csv
#grd_indices <- read.csv(grd_index_path)
# this data has some lines that have no labels, so we remove them 
grd_speclib <- grd_speclib[!is.na(grd_speclib$Functional_group1),]
head(grd_speclib)

,X,ScanID,Area,Code_name,Species_name,Functional_group1,Functional_group2,Species_name_Freq,Functional_group1_Freq,Functional_group2_Freq,⋯,Radiometric.Calibration,Units,Latitude,Longitude,Altitude,GPS.Time,Satellites,Calibrated.Reference.Correction.File,Channels,ScanNum
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
1,1,aleoch_Murph_061,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,2,aleoch_Murph_063,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,3,aleoch_Murph_064,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,4,aleoch_Murph_065,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
5,5,aleoch_Murph_066,Murphy,aleoch,Alectoria ochroleuca,Lichen,LightTerrestrialMacrolichen,6,453,118,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,6,alnfru_00003,Yukon_Delta,alnfru,Alnus sp.,ShrubDecid,ShrubAlder,82,360,82,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


In [4]:
img_base_path <- "Data/Ground_Validation/PFT_Image_spectra/PFT_Image_SpectralLib_Clean.csv"
img_speclib <- read.csv(img_base_path, header = TRUE)
# currently, not using the old pre-proccessing scheme and just doing it here.
#img_index_path <- Data/D_002_Image_SpecLib_Derivs.csv
#img_speclib <- read.csv(img_base_path)
head(img_speclib)

,X,UID,ScanNum,sample_name,PFT,FncGrp1,X398,X399,X400,X401,⋯,X990,X991,X992,X993,X994,X995,X996,X997,X998,X999
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,BisonGulchPFTsBetula1,1,spec_1,Betula,TreeBroadleaf,0.05243007,0.04516057,0.03909831,0.03482924,⋯,0.5636831,0.5717859,0.5632404,0.5485100,0.5380677,0.5400187,0.5561121,0.5870420,0.6335022,0.6961869
2,2,BisonGulchPFTsBetula1,1,spec_2,Betula,TreeBroadleaf,0.03280578,0.03279741,0.03278975,0.03278255,⋯,0.4652565,0.4655240,0.4657567,0.4659595,0.4661377,0.4662962,0.4664396,0.4665724,0.4666992,0.4668245
3,3,BisonGulchPFTsBetula1,1,spec_3,Betula,TreeBroadleaf,0.02415163,0.02445285,0.02475288,0.02505101,⋯,0.4713047,0.4704058,0.4696060,0.4689031,0.4682946,0.4677750,0.4673299,0.4669434,0.4665996,0.4662826
4,4,BisonGulchPFTsBetula1,1,spec_4,Betula,TreeBroadleaf,0.03013168,0.03041985,0.03070853,0.03097938,⋯,0.4282925,0.4317823,0.4380752,0.4476607,0.4610279,0.4783732,0.4991071,0.5225098,0.5478615,0.5744424
5,5,BisonGulchPFTsBetula1,1,spec_5,Betula,TreeBroadleaf,0.02798701,0.02818876,0.02838857,0.02858507,⋯,0.4344142,0.4353318,0.4362373,0.4371232,0.4379823,0.4388103,0.4396119,0.4403929,0.4411592,0.4419167
6,6,BisonGulchPFTsBetula1,1,spec_6,Betula,TreeBroadleaf,0.01771090,0.02832461,0.03569997,0.04027814,⋯,0.6276267,0.6381394,0.6008782,0.5358806,0.4632139,0.3988209,0.3475733,0.3125049,0.2966494,0.3030405


Okay, there are some metadata columns that should not be there for the next step - lets remove them with `subset`

In [6]:
img_bands <- subset(
    img_speclib, 
    select=-c(
        X,
    	UID,
        ScanNum,
    	sample_name,
    	PFT,
    	FncGrp1
    ))


grd_bands <- subset(
    grd_speclib, 
    select=-c(
        X,
        ScanID,
        Area,
        Code_name,
        Species_name,
        Functional_group1,
        Functional_group2,
        Species_name_Freq,
        Functional_group1_Freq,
        Functional_group2_Freq,
        Genus,
        Version,
        File.Name,
        Instrument,
        Detectors,
        Measurement,
        Date,
        Time,
        Battery.Voltage,
        Averages,
        Integration1,
        Integration2,
        Integration3,
        Dark.Mode,
        Foreoptic,
        Radiometric.Calibration,
        Units,
        Latitude,
        Longitude,
        Altitude,
        GPS.Time,
        Satellites,
        Calibrated.Reference.Correction.File,
        Channels,
        ScanNum
    )
)



Calculate the vegetation indices from the spectral libraries - its easy with lecospectR!

Note that the image-based scpectra are normalized from zero to one, and the ground specctra are on the range zero to one hundred.  

In [7]:
img_indices <- get_vegetation_indices(img_bands, NULL)# should have a default of NULL, you know?
grd_indices <- get_vegetation_indices(grd_bands, NULL)

In [8]:
write.csv(img_indices, file="Data/gs/x_train/img_indices_only.csv")

write.csv(grd_indices, file="Data/gs/x_train/grd_indices_only.csv")

In [7]:
head(img_indices)
head(img_indices)

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.008972526,0.009977190,0.4033027,2.606424,0.1919992,0.1823105,0.4513926,1.687375,0.10546899,1.230143,⋯,0.1858963,0.2555554,0.2937852,0.6583949,5.716883,27.50790,1.502310,-0.10251539,0.8407581,-0.1132324
2,0.007127460,0.008370667,0.4928890,3.283496,0.2205629,0.1740857,0.4311880,2.110946,0.10719907,1.103802,⋯,0.2102478,0.2862464,0.3001417,0.6959087,5.902966,27.53773,1.467985,-0.10410792,1.1292049,-0.1133074
3,0.006906736,0.008058805,0.5231705,3.940434,0.2402597,0.1788333,0.4415758,2.126987,0.10846255,1.065731,⋯,0.2137050,0.2948800,0.3062567,0.7350840,5.781993,27.43007,1.452730,-0.10563757,1.1222517,-0.1144615
4,0.008406272,0.009835800,0.3836683,2.971831,0.1742222,0.1636471,0.4133345,1.643268,0.10619240,1.165235,⋯,0.1815420,0.2471409,0.2856205,0.6047969,5.836640,28.45341,1.510704,-0.09834182,1.0664627,-0.1087631
5,0.006519263,0.007834586,0.5109378,3.397513,0.2156160,0.1586916,0.4100943,2.387455,0.09553940,1.056227,⋯,0.1987457,0.2606352,0.2755717,0.6306334,5.386860,27.10133,1.494922,-0.12274414,1.1404583,-0.1336822
6,0.008973633,0.009081469,0.3354300,2.065685,0.1621322,0.1601592,0.4241145,1.627772,0.09855188,1.589680,⋯,0.1656562,0.2234305,0.2613624,0.5515087,5.522945,27.15489,1.537855,-0.10372095,0.7253543,-0.1151156


,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.008972526,0.009977190,0.4033027,2.606424,0.1919992,0.1823105,0.4513926,1.687375,0.10546899,1.230143,⋯,0.1858963,0.2555554,0.2937852,0.6583949,5.716883,27.50790,1.502310,-0.10251539,0.8407581,-0.1132324
2,0.007127460,0.008370667,0.4928890,3.283496,0.2205629,0.1740857,0.4311880,2.110946,0.10719907,1.103802,⋯,0.2102478,0.2862464,0.3001417,0.6959087,5.902966,27.53773,1.467985,-0.10410792,1.1292049,-0.1133074
3,0.006906736,0.008058805,0.5231705,3.940434,0.2402597,0.1788333,0.4415758,2.126987,0.10846255,1.065731,⋯,0.2137050,0.2948800,0.3062567,0.7350840,5.781993,27.43007,1.452730,-0.10563757,1.1222517,-0.1144615
4,0.008406272,0.009835800,0.3836683,2.971831,0.1742222,0.1636471,0.4133345,1.643268,0.10619240,1.165235,⋯,0.1815420,0.2471409,0.2856205,0.6047969,5.836640,28.45341,1.510704,-0.09834182,1.0664627,-0.1087631
5,0.006519263,0.007834586,0.5109378,3.397513,0.2156160,0.1586916,0.4100943,2.387455,0.09553940,1.056227,⋯,0.1987457,0.2606352,0.2755717,0.6306334,5.386860,27.10133,1.494922,-0.12274414,1.1404583,-0.1336822
6,0.008973633,0.009081469,0.3354300,2.065685,0.1621322,0.1601592,0.4241145,1.627772,0.09855188,1.589680,⋯,0.1656562,0.2234305,0.2613624,0.5515087,5.522945,27.15489,1.537855,-0.10372095,0.7253543,-0.1151156


This is actually enough to start training models.  We have the vegetation indices, but instead of doing that, let's transform the data and write it to file.  Then we will proceed to creating the model corrections, etc.

In [8]:
img_resampled_bands <- resample_df(img_bands, drop_existing=TRUE)# corrects scale difference (poorly)
grd_resampled_bands <- resample_df(0.01*grd_bands, drop_existing=TRUE)
head(img_resampled_bands)
head(grd_resampled_bands)

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.03277507,0.03901295,0.03922547,0.03407865,0.03720464,0.03820831,0.03460818,0.03648937,0.03674930,0.03732257,⋯,0.6050174,0.4077844,0.3761586,0.5050814,0.3206515,0.4566141,0.5665918,0.3644928,0.4668932,0.5546659
2,0.03277141,0.03273556,0.03272566,0.03291429,0.03344372,0.03426495,0.03524766,0.03623183,0.03711415,0.03788855,⋯,0.4686437,0.4624061,0.4567163,0.4530922,0.4525675,0.4556402,0.4597004,0.4624739,0.4644677,0.4658803
3,0.02552067,0.02695883,0.02837947,0.02984791,0.03139014,0.03295136,0.03445546,0.03584613,0.03708338,0.03814974,⋯,0.5057955,0.5053712,0.5034267,0.5006850,0.4977061,0.4937533,0.4879560,0.4807059,0.4738716,0.4691781
4,0.03133775,0.03146211,0.03020504,0.02914408,0.03012126,0.03203688,0.03374192,0.03497939,0.03571163,0.03628934,⋯,0.5611057,0.5319484,0.4640202,0.4287151,0.4418199,0.4842263,0.4900493,0.4580981,0.4290783,0.4433673
5,0.02888800,0.02972499,0.03037238,0.03090010,0.03136556,0.03181339,0.03225852,0.03267638,0.03305330,0.03337770,⋯,0.4606015,0.4481342,0.4376910,0.4305552,0.4266213,0.4254519,0.4260657,0.4283058,0.4322131,0.4367651
6,0.04258530,0.03863350,0.03086016,0.03734187,0.03394887,0.02957398,0.03278345,0.02936776,0.03375559,0.03481238,⋯,0.5846634,0.5604421,0.4776229,0.6383636,0.3353291,0.5797247,0.7375400,0.4706714,0.4428437,0.5632284


,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.04786391,0.04478547,0.04214955,0.04121219,0.04261843,0.04609276,0.05056049,0.05609938,0.06384817,0.07352870,⋯,0.2659656,0.2671230,0.2678679,0.2684674,0.2692454,0.2704128,0.2720405,0.2738601,0.2755660,0.2769153
2,0.04820802,0.04623885,0.04461647,0.04424838,0.04589012,0.04933364,0.05350729,0.05836299,0.06480323,0.07251967,⋯,0.2141355,0.2152685,0.2162140,0.2170847,0.2180113,0.2190688,0.2202912,0.2216226,0.2229784,0.2242359
3,0.05849977,0.05496151,0.05154779,0.04986053,0.05175844,0.05731791,0.06499972,0.07520754,0.09011172,0.10881299,⋯,0.3184974,0.3190839,0.3198678,0.3205158,0.3208263,0.3212036,0.3219709,0.3229237,0.3238387,0.3246974
4,0.06252788,0.05817240,0.05399310,0.05201525,0.05424306,0.06092052,0.07068518,0.08364273,0.10123249,0.12223676,⋯,0.3418603,0.3421727,0.3423974,0.3425062,0.3425074,0.3425434,0.3427278,0.3430264,0.3434254,0.3440410
5,0.05055413,0.04732800,0.04457956,0.04357744,0.04533789,0.04989059,0.05639766,0.06534486,0.07839488,0.09474391,⋯,0.2914092,0.2920809,0.2927031,0.2931446,0.2934322,0.2940614,0.2953721,0.2970353,0.2986498,0.2999403
6,0.06056797,0.05514715,0.05001802,0.04665174,0.04565842,0.04598097,0.04511881,0.04293868,0.04154127,0.04142498,⋯,0.4787045,0.4774562,0.4759385,0.4748448,0.4745815,0.4744645,0.4739159,0.4731926,0.4726754,0.4726835


In [9]:
img_raw_with_na <- cbind(img_resampled_bands, img_indices)
grd_raw_with_na <- cbind(grd_resampled_bands, grd_indices)


In [10]:
img_raw <- impute_spectra(img_raw_with_na)
grd_raw <- impute_spectra(inf_to_na(grd_raw_with_na))# note also dropping an Inf (liekly div by 0 in veg index)

[1] "Imputing..."
[1] "Imputing..."


Apply the outlier transforms

In [11]:
grd_clipped <- clip_outliers(grd_raw)
grd_imputed <- impute_outliers_and_na(grd_raw)
grd_dropped <- grd_raw[detect_outliers_columnwise(grd_raw),]
img_clipped <- clip_outliers(img_raw)
img_imputed <- impute_outliers_and_na(img_raw)
img_dropped <- img_raw[detect_outliers_columnwise(img_raw),]

[1] "Imputing..."
[1] "Imputing..."
[1] "Imputing..."
[1] "Imputing..."


Now the center/scale transforms

In [12]:
grd_raw_robust <- columnwise_robust_scale(grd_raw)
img_raw_robust <- columnwise_robust_scale(img_raw)
grd_raw_minmax <- columnwise_min_max_scale(grd_raw)
img_raw_minmax <- columnwise_min_max_scale(img_raw)
grd_raw_standard <- standardize_df(grd_raw)
img_raw_standard <- standardize_df(img_raw)

grd_clipped_robust <- columnwise_robust_scale(grd_clipped)
grd_imputed_robust <- columnwise_robust_scale(grd_imputed)
grd_dropped_robust <- columnwise_robust_scale(grd_dropped)
img_clipped_robust <- columnwise_robust_scale(img_clipped)
img_imputed_robust <- columnwise_robust_scale(img_imputed)
img_dropped_robust <- columnwise_robust_scale(img_dropped)

grd_clipped_minmax <- columnwise_min_max_scale(grd_clipped)
grd_imputed_minmax <- columnwise_min_max_scale(grd_imputed)
grd_dropped_minmax <- columnwise_min_max_scale(grd_dropped)
img_clipped_minmax <- columnwise_min_max_scale(img_clipped)
img_imputed_minmax <- columnwise_min_max_scale(img_imputed)
img_dropped_minmax <- columnwise_min_max_scale(img_dropped)

grd_clipped_standard <- standardize_df(grd_clipped)
grd_imputed_standard <- standardize_df(grd_imputed)
grd_dropped_standard <- standardize_df(grd_imputed)
img_clipped_standard <- standardize_df(img_clipped)
img_imputed_standard <- standardize_df(img_imputed)
img_dropped_standard <- standardize_df(img_dropped)


Now, let's save all these data to disk

In [13]:
BASE_PATH <- "Data/gs/"
X_TRAIN_PATH <- paste0(BASE_PATH, "x_train/")
Y_TRAIN_PATH <- paste0(BASE_PATH, "y_train/")

X_TEST_PATH <- paste0(BASE_PATH, "x_test/")
Y_TEST_PATH <- paste0(BASE_PATH, "y_test/")

if(!dir.exists(BASE_PATH)){
    dir.create(BASE_PATH)
}
if(!dir.exists(X_TRAIN_PATH)){
    dir.create(X_TRAIN_PATH)
}
if(!dir.exists(Y_TRAIN_PATH)){
    dir.create(Y_TRAIN_PATH)
}
if(!dir.exists(X_TEST_PATH)){
    dir.create(X_TEST_PATH)
}
if(!dir.exists(Y_TEST_PATH)){
    dir.create(Y_TEST_PATH)
}


In [14]:
write.csv(grd_clipped, file=paste0(X_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_clipped_minmax, file=paste0(X_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_clipped_robust, file=paste0(X_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_clipped_standard, file=paste0(X_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_imputed, file=paste0(X_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_imputed_minmax, file=paste0(X_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_imputed_robust, file=paste0(X_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_imputed_standard, file=paste0(X_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_dropped, file=paste0(X_TRAIN_PATH, "grd_dropped_raw.csv"))
write.csv(grd_dropped_minmax, file=paste0(X_TRAIN_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_dropped_robust, file=paste0(X_TRAIN_PATH, "grd_dropped_robust.csv"))
write.csv(grd_dropped_standard, file=paste0(X_TRAIN_PATH, "grd_dropped_standard.csv"))

write.csv(grd_raw, file=paste0(X_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_raw_minmax, file=paste0(X_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_raw_robust, file=paste0(X_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_raw_standard, file=paste0(X_TRAIN_PATH, "grd_raw_standard.csv"))

In [15]:
write.csv(img_clipped, file=paste0(X_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax, file=paste0(X_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust, file=paste0(X_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard, file=paste0(X_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed, file=paste0(X_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax, file=paste0(X_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust, file=paste0(X_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard, file=paste0(X_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_raw, file=paste0(X_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax, file=paste0(X_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust, file=paste0(X_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard, file=paste0(X_TRAIN_PATH, "img_raw_standard.csv"))

## Labels for the above Data

In [7]:
img_targets <- img_speclib$FncGrp1 %>% as.factor()
grd_targets <- grd_speclib$Functional_group1 %>% as.factor()

In [12]:
write.csv(img_targets, file="Data/gs/y_train/img_indices_only.csv")
write.csv(impute_spectra(grd_targets), file="Data/gs/y_train/grd_indices_only.csv")


In [17]:
img_targets %>% table()

.
       Abiotic      Graminoid         Lichen           Moss     ShrubDecid 
           816            145             94             89            117 
ShrubEvergreen  TreeBroadleaf    TreeConifer 
           198            132           3091 

In [18]:
grd_targets %>% table()

.
       Abiotic           Forb      Graminoid         Lichen           Moss 
            94            158            112            417            122 
    ShrubDecid ShrubEvergreen  TreeBroadleaf    TreeConifer 
           297            105             21             17 

In [19]:
# drop entries with outliers to match training data
img_targets_dropped <- img_targets[detect_outliers_columnwise(img_raw)]
grd_targets_dropped <- grd_targets[detect_outliers_columnwise(grd_raw)]

In [20]:
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_clipped_standard.csv"))

write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_imputed_standard.csv"))

write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_raw.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_minmax.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_robust.csv"))
write.csv(grd_targets, file=paste0(Y_TRAIN_PATH, "grd_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TRAIN_PATH, "grd_dropped_standard.csv"))

In [21]:
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed, file=paste0(Y_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_targets, file=paste0(Y_TRAIN_PATH, "img_raw_standard.csv"))

write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_targets_dropped, file=paste0(Y_TRAIN_PATH, "img_dropped_standard.csv"))

## Test Data

Build the test data, and save it with the same names as the training data

In [22]:
set.seed(61718L)

permutation <-  permute::shuffle(length(img_targets))
sample <- create_stratified_sample(
    img_targets, 
    permutation = permutation,
    samples_per_pft = 15)
# split the data based on the above sample
img_targets_test <- img_targets[permutation][sample]
img_targets_train <- img_targets[permutation][-sample]
img_raw_test <- img_raw[permutation,][sample,]
img_raw_train <- img_raw[permutation,][-sample,]


[1] "Lichen"         "TreeConifer"    "TreeConifer"    "ShrubEvergreen"
[5] "TreeConifer"   


In [23]:
img_targets_test %>% as.factor() %>% table()

.
       Abiotic      Graminoid         Lichen           Moss     ShrubDecid 
            15             15             15             15             15 
ShrubEvergreen  TreeBroadleaf    TreeConifer 
            15             15             15 

In [24]:
# create the subsampled data and save them for each processing type/treatment

# clipped
img_clipped_train <- img_clipped[permutation,][-sample,]
img_clipped_test <- img_clipped[permutation,][sample,]
img_clipped_minmax_train <- img_clipped_minmax[permutation,][-sample,]
img_clipped_minmax_test <- img_clipped_minmax[permutation,][sample,]
img_clipped_robust_train <- img_clipped_robust[permutation,][-sample,]
img_clipped_robust_test <- img_clipped_robust[permutation,][sample,]
img_clipped_standard_train <- img_clipped_standard[permutation,][-sample]
img_clipped_standard_test <- img_clipped_standard[permutation,][sample,]

# raw (note one is done in the previous cell)
img_raw_minmax_train <- img_raw_minmax[permutation,][-sample,]
img_raw_minmax_test <- img_raw_minmax[permutation,][sample,]
img_raw_robust_train <- img_raw_robust[permutation,][-sample,]
img_raw_robust_test <- img_raw_robust[permutation,][sample,]
img_raw_standard_train <- img_raw_standard[permutation,][sample,]
img_raw_standard_test <- img_raw_standard[permutation,][sample,]

#imputed
img_imputed_train <- img_imputed[permutation,][-sample,]
img_imputed_test <- img_imputed[permutation,][sample,]
img_imputed_minmax_train <- img_imputed_minmax[permutation,][-sample,]
img_imputed_minmax_test <- img_imputed_minmax[permutation,][sample,]
img_imputed_robust_train <- img_imputed_robust[permutation,][-sample,]
img_imputed_robust_test <- img_imputed_robust[permutation,][sample,]
img_imputed_standard_train <- img_imputed_standard[permutation,][-sample,]
img_imputed_standard_test <- img_imputed_standard[permutation,][sample,]



In [25]:
print(length(img_targets_test))
print(nrow(img_clipped_robust_test))

[1] 120
[1] 120


### Image-based Training Data

In [26]:
write.csv(img_clipped_train, file=paste0(X_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax_train, file=paste0(X_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust_train, file=paste0(X_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard_train, file=paste0(X_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed_train, file=paste0(X_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax_train, file=paste0(X_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust_train, file=paste0(X_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard_train, file=paste0(X_TRAIN_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_raw_train, file=paste0(X_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax_train, file=paste0(X_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust_train, file=paste0(X_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard_train, file=paste0(X_TRAIN_PATH, "img_raw_standard.csv"))

In [27]:
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_clipped_standard.csv"))

write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_imputed_standard.csv"))

#write.csv(img_dropped, file=paste0(X_TRAIN_PATH, "img_dropped_raw.csv"))
#write.csv(img_dropped_minmax, file=paste0(X_TRAIN_PATH, "img_dropped_minmax.csv"))
#write.csv(img_dropped_robust, file=paste0(X_TRAIN_PATH, "img_dropped_robust.csv"))
#write.csv(img_dropped_standard, file=paste0(X_TRAIN_PATH, "img_dropped_standard.csv"))

write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_raw.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_minmax.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_robust.csv"))
write.csv(img_targets_train, file=paste0(Y_TRAIN_PATH, "img_raw_standard.csv"))

### Image Based Test Data
Note: this image-based test set is used for all the models (ground included)

In [28]:
write.csv(img_clipped_test, file=paste0(X_TEST_PATH, "img_clipped_raw.csv"))
write.csv(img_clipped_minmax_test, file=paste0(X_TEST_PATH, "img_clipped_minmax.csv"))
write.csv(img_clipped_robust_test, file=paste0(X_TEST_PATH, "img_clipped_robust.csv"))
write.csv(img_clipped_standard_test, file=paste0(X_TEST_PATH, "img_clipped_standard.csv"))

write.csv(img_imputed_test, file=paste0(X_TEST_PATH, "img_imputed_raw.csv"))
write.csv(img_imputed_minmax_test, file=paste0(X_TEST_PATH, "img_imputed_minmax.csv"))
write.csv(img_imputed_robust_test, file=paste0(X_TEST_PATH, "img_imputed_robust.csv"))
write.csv(img_imputed_standard_test, file=paste0(X_TEST_PATH, "img_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TEST_PATH, "img_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TEST_PATH, "img_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TEST_PATH, "img_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TEST_PATH, "img_dropped_standard.csv"))

write.csv(img_raw_test, file=paste0(X_TEST_PATH, "img_raw_raw.csv"))
write.csv(img_raw_minmax_test, file=paste0(X_TEST_PATH, "img_raw_minmax.csv"))
write.csv(img_raw_robust_test, file=paste0(X_TEST_PATH, "img_raw_robust.csv"))
write.csv(img_raw_standard_test, file=paste0(X_TEST_PATH, "img_raw_standard.csv"))

### Ground test (from the images)

In [29]:
write.csv(img_clipped_test, file=paste0(X_TEST_PATH, "grd_clipped_raw.csv"))
write.csv(img_clipped_minmax_test, file=paste0(X_TEST_PATH, "grd_clipped_minmax.csv"))
write.csv(img_clipped_robust_test, file=paste0(X_TEST_PATH, "grd_clipped_robust.csv"))
write.csv(img_clipped_standard_test, file=paste0(X_TEST_PATH, "grd_clipped_standard.csv"))

write.csv(img_imputed_test, file=paste0(X_TEST_PATH, "grd_imputed_raw.csv"))
write.csv(img_imputed_minmax_test, file=paste0(X_TEST_PATH, "grd_imputed_minmax.csv"))
write.csv(img_imputed_robust_test, file=paste0(X_TEST_PATH, "grd_imputed_robust.csv"))
write.csv(img_imputed_standard_test, file=paste0(X_TEST_PATH, "grd_imputed_standard.csv"))

write.csv(img_dropped, file=paste0(X_TEST_PATH, "grd_dropped_raw.csv"))
write.csv(img_dropped_minmax, file=paste0(X_TEST_PATH, "grd_dropped_minmax.csv"))
write.csv(img_dropped_robust, file=paste0(X_TEST_PATH, "grd_dropped_robust.csv"))
write.csv(img_dropped_standard, file=paste0(X_TEST_PATH, "grd_dropped_standard.csv"))

write.csv(img_raw_test, file=paste0(X_TEST_PATH, "grd_raw_raw.csv"))
write.csv(img_raw_minmax_test, file=paste0(X_TEST_PATH, "grd_raw_minmax.csv"))
write.csv(img_raw_robust_test, file=paste0(X_TEST_PATH, "grd_raw_robust.csv"))
write.csv(img_raw_standard_test, file=paste0(X_TEST_PATH, "grd_raw_standard.csv"))

In [30]:
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_clipped_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_imputed_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "grd_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "grd_dropped_standard.csv"))

In [31]:
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_clipped_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_imputed_standard.csv"))

write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_raw.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_minmax.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_robust.csv"))
write.csv(img_targets_test, file=paste0(Y_TEST_PATH, "img_raw_standard.csv"))

write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_raw.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_minmax.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_robust.csv"))
write.csv(grd_targets_dropped, file=paste0(Y_TEST_PATH, "img_dropped_standard.csv"))

In [54]:
# need to write the targets fror training
clip_transform <- create_clip_transform(
    img_raw
)

Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list(upper = upper_fence, lower = lower_fence):
“number of items to replace is not a multiple of replacement length”
Warning message in fences[column_name] <- list

In [55]:
save(clip_transform, file="./mle/clip_transform.rda")

In [56]:
clipped_2 <- clip_transform(img_raw)# clipped 2

## Sensor Correction

In this section, we create the models (and do some data transforms) to make the sensor-correction models and create the corrected data (only three times).  

We do this first for the raw (including outliers) data.

In [33]:
grd_resampled_to_match_img_bands <- resample_df(
    grd_bands,
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)
head(grd_resampled_to_match_img_bands)
head(img_bands)

,X398_5nm,X399_5nm,X400_5nm,X401_5nm,X402_5nm,X403_5nm,X404_5nm,X405_5nm,X406_5nm,X407_5nm,⋯,X990_5nm,X991_5nm,X992_5nm,X993_5nm,X994_5nm,X995_5nm,X996_5nm,X997_5nm,X998_5nm,X999_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,4.8846,4.8925,4.9184,4.9628,4.9102,4.7963,4.6673,4.5396,4.4311,4.3730,⋯,27.6272,27.6523,27.6757,27.6977,27.7181,27.7370,27.7544,27.7703,27.7846,27.7974
2,4.9516,4.9430,4.9209,4.8555,4.8264,4.7953,4.7073,4.6387,4.5811,4.5128,⋯,22.3594,22.3834,22.4069,22.4301,22.4529,22.4753,22.4974,22.5191,22.5404,22.5613
3,6.0398,6.0197,6.0054,5.9880,5.9352,5.8390,5.6852,5.5654,5.4625,5.3369,⋯,32.4257,32.4426,32.4593,32.4758,32.4922,32.5085,32.5246,32.5405,32.5563,32.5719
4,6.4706,6.4441,6.4293,6.4678,6.3939,6.2293,5.9987,5.8117,5.6694,5.5691,⋯,34.3678,34.3810,34.3951,34.4103,34.4264,34.4435,34.4616,34.4807,34.5008,34.5219
5,5.2403,5.2240,5.2112,5.1750,5.1250,5.0416,4.8941,4.7927,4.7200,4.6447,⋯,29.9326,29.9557,29.9775,29.9979,30.0170,30.0348,30.0513,30.0665,30.0803,30.0928
6,6.3948,6.3681,6.3422,6.3502,6.2172,5.9899,5.7240,5.5414,5.4176,5.3077,⋯,47.2582,47.2609,47.2660,47.2734,47.2832,47.2953,47.3097,47.3265,47.3457,47.3671


,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X990,X991,X992,X993,X994,X995,X996,X997,X998,X999
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.05243007,0.04516057,0.03909831,0.03482924,0.03285911,0.03287741,0.03409723,0.03572572,0.03711334,0.03818382,⋯,0.5636831,0.5717859,0.5632404,0.5485100,0.5380677,0.5400187,0.5561121,0.5870420,0.6335022,0.6961869
2,0.03280578,0.03279741,0.03278975,0.03278255,0.03277556,0.03276862,0.03276165,0.03275454,0.03274727,0.03274001,⋯,0.4652565,0.4655240,0.4657567,0.4659595,0.4661377,0.4662962,0.4664396,0.4665724,0.4666992,0.4668245
3,0.02415163,0.02445285,0.02475288,0.02505101,0.02534659,0.02563954,0.02593011,0.02621855,0.02650515,0.02679029,⋯,0.4713047,0.4704058,0.4696060,0.4689031,0.4682946,0.4677750,0.4673299,0.4669434,0.4665996,0.4662826
4,0.03013168,0.03041985,0.03070853,0.03097938,0.03121466,0.03140237,0.03153387,0.03160058,0.03159647,0.03152571,⋯,0.4282925,0.4317823,0.4380752,0.4476607,0.4610279,0.4783732,0.4991071,0.5225098,0.5478615,0.5744424
5,0.02798701,0.02818876,0.02838857,0.02858507,0.02877691,0.02896310,0.02914283,0.02931529,0.02947982,0.02963626,⋯,0.4344142,0.4353318,0.4362373,0.4371232,0.4379823,0.4388103,0.4396119,0.4403929,0.4411592,0.4419167
6,0.01771090,0.02832461,0.03569997,0.04027814,0.04251483,0.04301360,0.04246431,0.04155789,0.04080908,0.04002789,⋯,0.6276267,0.6381394,0.6008782,0.5358806,0.4632139,0.3988209,0.3475733,0.3125049,0.2966494,0.3030405


In [34]:
colnames(grd_resampled_to_match_img_bands) <- colnames(img_bands)

In [35]:
grd_resampled_to_match_img_bands$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

In [36]:
matched_data <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_match_img_bands,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)
head(matched_data$left)
head(matched_data$right)

,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
282,0.07475367,0.07481203,0.07487065,0.07492971,0.07498942,0.07504996,0.07511153,0.07517430,0.07523847,0.07530419,⋯,0.1400329,0.13983717,0.13964177,0.13944661,0.13925159,0.1390567,0.1388620,0.1386674,0.1384729,Abiotic
283,0.07782648,0.07782431,0.07782241,0.07782099,0.07782029,0.07782054,0.07782201,0.07782498,0.07782970,0.07783637,⋯,0.1436408,0.14356986,0.14349957,0.14342999,0.14336117,0.1432931,0.1432257,0.1431591,0.1430930,Abiotic
284,0.10539039,0.09728302,0.09015593,0.08423040,0.07969948,0.07646851,0.07427497,0.07285423,0.07195485,0.07137823,⋯,0.0698050,0.07328251,0.07861538,0.08638126,0.09667979,0.1083275,0.1199279,0.1300846,0.1374010,Abiotic
285,0.06819310,0.06845060,0.06870810,0.06896560,0.06922310,0.06948061,0.06973811,0.06999562,0.07025313,0.07051065,⋯,0.1733476,0.17341998,0.17349240,0.17356485,0.17363732,0.1737098,0.1737823,0.1738548,0.1739273,Abiotic
286,0.07344760,0.07374939,0.07405118,0.07435297,0.07465477,0.07495658,0.07525839,0.07556022,0.07586206,0.07616392,⋯,0.1509956,0.15092268,0.15084978,0.15077688,0.15070398,0.1506311,0.1505582,0.1504853,0.1504124,Abiotic
287,0.07411054,0.07436906,0.07462761,0.07488617,0.07514477,0.07540341,0.07566211,0.07592088,0.07617973,0.07643868,⋯,0.1371022,0.13695218,0.13680213,0.13665208,0.13650203,0.1363520,0.1362019,0.1360519,0.1359019,Abiotic


,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
107,12.5296,12.6062,12.6908,12.7803,12.7955,12.7510,12.6624,12.6126,12.5953,12.6003,⋯,18.4181,18.4210,18.4242,18.4277,18.4316,18.4358,18.4403,18.4452,18.4504,Abiotic
108,8.0423,8.0503,8.0688,8.1035,8.0919,8.0430,7.9695,7.8667,7.7654,7.7172,⋯,18.4996,18.5051,18.5103,18.5152,18.5197,18.5240,18.5279,18.5316,18.5349,Abiotic
109,7.8445,7.8459,7.8326,7.7877,7.7383,7.6777,7.5947,7.5238,7.4640,7.4080,⋯,21.0692,21.0950,21.1205,21.1458,21.1709,21.1956,21.2202,21.2445,21.2685,Abiotic
110,7.0172,7.0003,6.9872,7.0021,7.0185,7.0160,6.9667,6.9245,6.8957,6.8962,⋯,13.6072,13.6251,13.6433,13.6617,13.6805,13.6995,13.7188,13.7383,13.7581,Abiotic
111,6.4009,6.3791,6.3679,6.4081,6.3689,6.2896,6.2227,6.1535,6.1086,6.1448,⋯,17.2888,17.3038,17.3191,17.3347,17.3505,17.3666,17.3830,17.3996,17.4165,Abiotic
112,5.8715,5.9002,5.9289,5.9330,5.9219,5.8927,5.8412,5.7605,5.6801,5.6494,⋯,14.2387,14.2526,14.2674,14.2831,14.2998,14.3173,14.3358,14.3551,14.3754,Abiotic


In [37]:
correction_model <- build_columnwise_sensor_correction_model(
    matched_data$left,
    matched_data$right,
    grouping_variables =c("targets","targets")
)


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.020462 -0.009043 -0.005602  0.008793  0.049934 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.945e-02  6.842e-04   28.42   <2e-16 ***
right_vec   1.417e-03  5.023e-05   28.22   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01339 on 647 degrees of freedom
Multiple R-squared:  0.5517,	Adjusted R-squared:  0.551 
F-statistic: 796.3 on 1 and 647 DF,  p-value: < 2.2e-16


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.020201 -0.008933 -0.005806  0.008697  0.037070 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.976e-02  6.700e-04   29.50   <2e-16 ***
right_vec   1.394e-03  4.925e-05   28.31   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 

In [38]:
print(correction_model)

$X398

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.019447     0.001417  


$X399

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.019763     0.001394  


$X400

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.020037     0.001379  


$X401

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.020277     0.001369  


$X402

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.020614     0.001357  


$X403

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.021028     0.001343  


$X404

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.021458     0.001331  


$X405

Call:
lm(formula = left_vec ~ right_vec)

Coefficients:
(Intercept)    right_vec  
   0.021832     0.001322  


$X406

Call:
lm(formula = left_vec ~ right_vec)


In [39]:
grd_corrected_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
head(grd_corrected_bands)

[1] "Correcting X398"
[1] "Correcting X399"
[1] "Correcting X400"
[1] "Correcting X401"
[1] "Correcting X402"
[1] "Correcting X403"
[1] "Correcting X404"
[1] "Correcting X405"
[1] "Correcting X406"
[1] "Correcting X407"
[1] "Correcting X408"
[1] "Correcting X409"
[1] "Correcting X410"
[1] "Correcting X411"
[1] "Correcting X412"
[1] "Correcting X413"
[1] "Correcting X414"
[1] "Correcting X415"
[1] "Correcting X416"
[1] "Correcting X417"
[1] "Correcting X418"
[1] "Correcting X419"
[1] "Correcting X420"
[1] "Correcting X421"
[1] "Correcting X422"
[1] "Correcting X423"
[1] "Correcting X424"
[1] "Correcting X425"
[1] "Correcting X426"
[1] "Correcting X427"
[1] "Correcting X428"
[1] "Correcting X429"
[1] "Correcting X430"
[1] "Correcting X431"
[1] "Correcting X432"
[1] "Correcting X433"
[1] "Correcting X434"
[1] "Correcting X435"
[1] "Correcting X436"
[1] "Correcting X437"
[1] "Correcting X438"
[1] "Correcting X439"
[1] "Correcting X440"
[1] "Correcting X441"
[1] "Correcting X442"
[1] "Corre

,X398,X399,X400,X401,X402,X403,X404,X405,X406,X407,⋯,X991,X992,X993,X994,X995,X996,X997,X998,X999,targets
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,0.02637021,0.02658293,0.02681803,0.02706952,0.02727871,0.02747130,0.02766833,0.02783201,0.02797021,0.02809886,⋯,0.1818062,0.1793266,0.1753943,0.1723933,0.1705940,0.1675363,0.1624677,0.1571582,0.1503836,Lichen
2,0.02646518,0.02665332,0.02682148,0.02692266,0.02716497,0.02746996,0.02772156,0.02796299,0.02816750,0.02828223,⋯,0.1604729,0.1582023,0.1540679,0.1509742,0.1491923,0.1455509,0.1391044,0.1320851,0.1229338,Lichen
3,0.02800764,0.02815423,0.02831664,0.02847262,0.02866994,0.02887215,0.02902279,0.02918780,0.02932673,0.02936319,⋯,0.2012016,0.1985055,0.1947388,0.1918145,0.1900019,0.1874858,0.1836910,0.1799722,0.1754134,Lichen
4,0.02861827,0.02874584,0.02890105,0.02912929,0.02929254,0.02939651,0.02943994,0.02951333,0.02959885,0.02966776,⋯,0.2090500,0.2062667,0.2025708,0.1996830,0.1978724,0.1955866,0.1923232,0.1892690,0.1856361,Lichen
5,0.02687439,0.02704503,0.02722170,0.02735994,0.02757026,0.02780086,0.02797012,0.02816653,0.02835018,0.02845524,⋯,0.1911324,0.1885552,0.1847068,0.1817453,0.1799402,0.1771422,0.1726838,0.1681342,0.1624170,Lichen
6,0.02851083,0.02863990,0.02878097,0.02896834,0.02905270,0.02907488,0.02907441,0.02915608,0.02926768,0.02932488,⋯,0.2611994,0.2578702,0.2546483,0.2519850,0.2501465,0.2493188,0.2494758,0.2506818,0.2529758,ShrubDecid


In [40]:
grd_corrected_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
head(grd_corrected_indices)

,Boochs,Boochs2,CARI,Carter,Carter2,Carter3,Carter4,Carter5,Carter6,CI,⋯,TCARI,TCARIOSAVI,TCARI2,TCARI2OSAVI2,TGI,TVI,Vogelmann,Vogelmann2,Vogelmann3,Vogelmann4
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.002381295,0.001910892,0.2635879,3.564008,0.5314961,0.4046933,0.7028920,1.285383,0.07577025,1.012063,⋯,0.06580557,0.2068874,0.09152356,0.5725180,1.544914,6.379498,1.138300,-0.02961013,0.9822749,-0.03052389
2,0.002549777,0.001763978,0.2511120,3.189676,0.5215856,0.4021622,0.7183551,1.335906,0.07228484,1.016968,⋯,0.07289687,0.2264193,0.08482058,0.5533605,1.699010,6.446032,1.118845,-0.02344562,0.9072289,-0.02412762
3,0.002318314,0.002211095,0.2806582,3.833218,0.5326770,0.4297509,0.6817851,1.212861,0.08782017,1.007785,⋯,0.05842626,0.1881597,0.11232708,0.6444097,1.528353,6.704626,1.169424,-0.03717324,1.0913310,-0.03847743
4,0.002312770,0.002290771,0.2841312,3.879377,0.5272567,0.4293129,0.6715073,1.199209,0.09087412,1.005452,⋯,0.05697684,0.1825453,0.11899829,0.6634249,1.533878,6.861560,1.177691,-0.03938247,1.1325660,-0.04080856
5,0.002387488,0.002132143,0.2724334,3.759951,0.5274929,0.4215376,0.6871894,1.230462,0.08364653,1.007648,⋯,0.06069181,0.1941528,0.10592594,0.6213191,1.554349,6.618392,1.159009,-0.03414246,1.0438457,-0.03530157
6,0.004017585,0.004032547,0.2761194,2.713578,0.2928350,0.2124903,0.4983470,1.620458,0.06907974,1.017620,⋯,0.10888868,0.1991132,0.17845540,0.5611980,2.725589,13.745792,1.375279,-0.08565282,1.0138187,-0.09157656


In [41]:
grd_corrected_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)
head(grd_corrected_resampled_bands)

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



,X402.593_5nm,X407.593_5nm,X412.593_5nm,X417.593_5nm,X422.593_5nm,X427.593_5nm,X432.593_5nm,X437.593_5nm,X442.593_5nm,X447.593_5nm,⋯,X947.593_5nm,X952.593_5nm,X957.593_5nm,X962.593_5nm,X967.593_5nm,X972.593_5nm,X977.593_5nm,X982.593_5nm,X987.593_5nm,X992.593_5nm
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.02740206,0.02815305,0.02854418,0.02909697,0.02974769,0.03054028,0.03162673,0.03290461,0.03466404,0.03653387,⋯,0.1802482,0.1767299,0.1790706,0.1780562,0.1818159,0.1855247,0.1846986,0.1854914,0.1820047,0.1773957
2,0.02732942,0.02836598,0.02889343,0.02955696,0.03015937,0.03099921,0.03208175,0.03326427,0.03476213,0.03639913,⋯,0.1608380,0.1575138,0.1606400,0.1587550,0.1627403,0.1654892,0.1640014,0.1639235,0.1588322,0.1562669
3,0.02879086,0.02942490,0.02992718,0.03026564,0.03100644,0.03215771,0.03375261,0.03578497,0.03858237,0.04187252,⋯,0.1999480,0.1959790,0.1975312,0.1977461,0.2009720,0.2052900,0.2047446,0.2057211,0.2032627,0.1965706
4,0.02935773,0.02975621,0.03032106,0.03062475,0.03125250,0.03268986,0.03463188,0.03701450,0.04024667,0.04392572,⋯,0.2086859,0.2043948,0.2057979,0.2058916,0.2089897,0.2136916,0.2130444,0.2140075,0.2118787,0.2043295
5,0.02769439,0.02849479,0.02887262,0.02949647,0.03003152,0.03113648,0.03244466,0.03436734,0.03677789,0.03975656,⋯,0.1898395,0.1859246,0.1879118,0.1873708,0.1908078,0.1947937,0.1939209,0.1951469,0.1921639,0.1866240
6,0.02906247,0.02937925,0.02974373,0.02989992,0.03007971,0.03048701,0.03090246,0.03099385,0.03122152,0.03160626,⋯,0.2600086,0.2544651,0.2533717,0.2558402,0.2578774,0.2653806,0.2654268,0.2677968,0.2687752,0.2559211


In [42]:
write.csv(
    cbind(grd_corrected_resampled_bands, grd_corrected_indices), 
    file=paste0(X_TRAIN_PATH, "grd_raw_corrected.csv")
    )

# save labels also
write.csv(
    grd_resampled_to_match_img_bands$targets,
    file=paste0(Y_TRAIN_PATH, "grd_raw_corrected.csv"))

In [11]:
write.csv(grd_targets, file="grd_raw_corrected.csv")


Now that that is done, we will move on to the clipped data

In [43]:
grd_resampled_to_img_clipped <- resample_df(
    clip_outliers(grd_bands),
    min_wavelength = 398,
    max_wavelength = 999,
    delta=1,
    drop_existing = TRUE
)

colnames(grd_resampled_to_img_clipped) <- colnames(img_bands)

grd_resampled_to_img_clipped$targets <- grd_targets
img_bands_with_targets <- img_bands
img_bands_with_targets$targets <- img_targets

matched_data_clipped <- create_matched_data(
    img_bands_with_targets,
    grd_resampled_to_img_clipped,
    cols=c("targets","targets")# assumes joining on columns named "targets" in each data.frame
)

correction_model <- build_columnwise_sensor_correction_model(
    matched_data_clipped$left,
    matched_data_clipped$right
)
grd_corrected_clipped_bands <- apply_sensor_correction_model(
    correction_model,
    grd_resampled_to_match_img_bands,
    ignore_cols=c("targets")
)
grd_corrected_clipped_indices <- get_vegetation_indices(grd_corrected_bands, NULL)
grd_corrected_clipped_resampled_bands <- resample_df(grd_corrected_bands, drop_existing=TRUE)





Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.034182 -0.012948 -0.006233  0.012154  0.072142 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.0213422  0.0012650   16.87   <2e-16 ***
right_vec   0.0015178  0.0001498   10.13   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.01857 on 647 degrees of freedom
Multiple R-squared:  0.1369,	Adjusted R-squared:  0.1355 
F-statistic: 102.6 on 1 and 647 DF,  p-value: < 2.2e-16


Call:
lm(formula = left_vec ~ right_vec)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.032167 -0.012542 -0.005969  0.011554  0.063867 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.0211284  0.0012323   17.15   <2e-16 ***
right_vec   0.0015661  0.0001467   10.68   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error:

Using spline to predict value at new bands...

Beware the spectra are now partially smoothed.



In [44]:
write.csv(
    cbind(
        grd_corrected_clipped_indices,
        grd_corrected_clipped_resampled_bands
    ),
    file=paste0(
        X_TRAIN_PATH,
        "grd_clipped_corrected.csv"
    )
)

# save labels also
write.csv(
    grd_resampled_to_match_img_bands$targets,
    file=paste0(Y_TRAIN_PATH, "grd_clipped_corrected.csv"))

And finally the dropped outlier one

notes for later: should probably try PCA here.  clip -> scale -> PCA -> subset (and scale again for models like SVM and kNN)

...to be continued